In [1]:
#참고링크 https://github.com/allenai/bilm-tf
#참고링크 https://appliedmachinelearning.wordpress.com/2019/11/30/training-elmo-from-scratch-on-custom-data-set-for-generating-embeddings-tensorflow/

In [ ]:
# #cpu 사용
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 패키지 임포트

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
import string
import nltk
import pickle
import json
sys.path.append('C:/Users/User/Desktop/20Newsgroups')
import tensorflow as tf
import scipy.spatial.distance as ds
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from collections import Counter
from data_preprocessing import *

C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\anaconda3\envs\elem\lib\site-packages\tensorflow\python\frame

# 데이터 로드

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

# 전처리

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [5]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [6]:
x_train=list(X_train)
x_test=list(X_test)

In [7]:
pattern = '[^a-z.0-9 ]'
clr_x_trian=[]
clr_x_test=[]
for sen in x_train:
    clr_x_trian.append(re.sub(pattern, '', sen))
for sen in x_test:
    clr_x_test.append(re.sub(pattern, '', sen))

In [8]:
x_train=clr_x_trian
x_test=clr_x_test

In [9]:
to_txt_filter=x_train+x_test

In [10]:
y=list(y_train)+list(y_test)

# 사전 학습 ELMo 불러오기

In [11]:
#사전 학습된 모델이 있는 디렉토리
datadir = 'elmo_model(20ng256)'
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'swb_weights.hdf5')
 
batcher = Batcher(vocab_file, 50)
 
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
context_embeddings_op = bilm(context_character_ids)
 
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

#테스트 임베딩
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
 
tokenized_context = [sentence.split() for sentence in raw_context]
print(tokenized_context)

USING SKIP CONNECTIONS
[['Technology', 'has', 'advanced', 'so', 'much', 'in', 'new', 'scientific', 'world'], ['My', 'child', 'participated', 'in', 'fancy', 'dress', 'competition'], ['Fashion', 'industry', 'has', 'seen', 'tremendous', 'growth', 'in', 'new', 'designs']]


In [12]:
with tf.Session() as sess:    
    sess.run(tf.global_variables_initializer())     
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (3, 11, 50)
Shape of generated embeddings =  (3, 9, 256)


In [13]:
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

Euclidean Distance Comparison - 

Dress-Technology =  17.68 
Dress-Fashion =  14.36


Cosine Distance Comparison - 

Dress-Technology =  0.85 
Dress-Fashion =  0.83


# 문서 단위 임베딩

In [14]:
#메모리 오류로 임베딩 되지 않는 문서의 인덱스를 찾기위한 리스트
except_lst=[]

In [ ]:
for i in range(len(x_train)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_train[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./elmo_embedding/train(20ng256)/doc%d'%i,elmo_context_input_[0])
    except:
        except_lst.append(i)

Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 84, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 150, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 268, 256)
Shape of generated embeddings =  (1, 160, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 59, 256)
Shape of generated embeddings =  (1, 149, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 94, 256)


In [18]:
except_lst

[]

In [18]:
except_test_lst=[]

In [19]:
for i in range(len(x_test)):
    try:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            context_ids = batcher.batch_sentences([x_test[i].split()])
            elmo_context_input_ = sess.run(elmo_context_input['weighted_op'],feed_dict={context_character_ids: context_ids}) 
        print("Shape of generated embeddings = ",elmo_context_input_.shape)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./elmo_embedding/test(20ng256)/test%d'%i,elmo_context_input_[0])
    except:
        except_test_lst.append(i)

Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 65, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 248, 256)
Shape of generated embeddings =  (1, 83, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 33, 256)
Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 39, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 92, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 104, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings

Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 174, 256)
Shape of generated embeddings =  (1, 208, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 45, 256)
Shape of generated embeddings =  (1, 160, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 93, 256)
Shape of generated embeddings =  (1, 156, 256)
Shape of generated embeddings =  (1, 464, 256)
Shape of generated embeddings =  (1, 177, 256)
Shape of generated embeddings =  (1, 307, 256)
Shape of generated embeddings =  (1, 61, 256)
Shape of generated embeddings =  (1, 5005, 256)
Shape of generated embeddings =  (1, 16, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 90, 256)
Shape of generated embeddings =  (1, 250, 256)
Shape of generated embed

Shape of generated embeddings =  (1, 108, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 25, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 19, 256)
Shape of generated embeddings =  (1, 77, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 6616, 256)
Shape of generated embeddings =  (1, 110, 256)
Shape of generated embeddings =  (1, 79, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 36, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 99, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 31, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 180, 256)
Shape of generated embeddings =  (1, 119, 256)
Shape of generated embeddings =  (1, 107, 256)
Shape of generated embeddings =  (1, 212, 256)
Shape of generated embedd

Shape of generated embeddings =  (1, 63, 256)
Shape of generated embeddings =  (1, 143, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 72, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 67, 256)
Shape of generated embeddings =  (1, 382, 256)
Shape of generated embeddings =  (1, 60, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 46, 256)
Shape of generated embeddings =  (1, 158, 256)
Shape of generated embeddings =  (1, 164, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 87, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 21, 256)
Shape of generated embeddings =  (1, 215, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embedding

Shape of generated embeddings =  (1, 52, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 58, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 241, 256)
Shape of generated embeddings =  (1, 304, 256)
Shape of generated embeddings =  (1, 339, 256)
Shape of generated embeddings =  (1, 172, 256)
Shape of generated embeddings =  (1, 510, 256)
Shape of generated embeddings =  (1, 78, 256)
Shape of generated embeddings =  (1, 76, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddings =  (1, 296, 256)
Shape of generated embeddings =  (1, 206, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 92, 256)
Shape of generated embeddings =  (1, 104, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embedd

...

Shape of generated embeddings =  (1, 22, 256)
Shape of generated embeddings =  (1, 32, 256)
Shape of generated embeddings =  (1, 35, 256)
Shape of generated embeddings =  (1, 13, 256)
Shape of generated embeddings =  (1, 61, 256)
Shape of generated embeddings =  (1, 385, 256)
Shape of generated embeddings =  (1, 212, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 96, 256)
Shape of generated embeddings =  (1, 233, 256)
Shape of generated embeddings =  (1, 4813, 256)
Shape of generated embeddings =  (1, 50, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 121, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 43, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 10, 256)
Shape of generated embeddings =  (1, 390, 256)
Shape of generated embeddings =  (1, 169, 256)
Shape of generated embeddin

Shape of generated embeddings =  (1, 109, 256)
Shape of generated embeddings =  (1, 53, 256)
Shape of generated embeddings =  (1, 200, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 98, 256)
Shape of generated embeddings =  (1, 75, 256)
Shape of generated embeddings =  (1, 480, 256)
Shape of generated embeddings =  (1, 138, 256)
Shape of generated embeddings =  (1, 192, 256)
Shape of generated embeddings =  (1, 582, 256)
Shape of generated embeddings =  (1, 17, 256)
Shape of generated embeddings =  (1, 451, 256)
Shape of generated embeddings =  (1, 28, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 62, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 71, 256)
Shape of generated embeddings =  (1, 47, 256)
Shape of generated embeddings =  (1, 163, 256)
Shape of generated embeddings =  (1, 131, 256)
Shape of generated embeddings =  (1, 86, 256)
Shape of generated embedd

Shape of generated embeddings =  (1, 162, 256)
Shape of generated embeddings =  (1, 196, 256)
Shape of generated embeddings =  (1, 85, 256)
Shape of generated embeddings =  (1, 66, 256)
Shape of generated embeddings =  (1, 88, 256)
Shape of generated embeddings =  (1, 38, 256)
Shape of generated embeddings =  (1, 112, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 23, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 40, 256)
Shape of generated embeddings =  (1, 247, 256)
Shape of generated embeddings =  (1, 41, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 274, 256)
Shape of generated embeddings =  (1, 128, 256)
Shape of generated embeddings =  (1, 162, 256)
Shape of generated embeddings =  (1, 114, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 74, 256)
Shape of generated embeddings =  (1, 115, 256)
Shape of generated embed

Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 94, 256)
Shape of generated embeddings =  (1, 92, 256)
Shape of generated embeddings =  (1, 635, 256)
Shape of generated embeddings =  (1, 171, 256)
Shape of generated embeddings =  (1, 129, 256)
Shape of generated embeddings =  (1, 26, 256)
Shape of generated embeddings =  (1, 157, 256)
Shape of generated embeddings =  (1, 54, 256)
Shape of generated embeddings =  (1, 141, 256)
Shape of generated embeddings =  (1, 64, 256)
Shape of generated embeddings =  (1, 1422, 256)
Shape of generated embeddings =  (1, 116, 256)
Shape of generated embeddings =  (1, 55, 256)
Shape of generated embeddings =  (1, 27, 256)
Shape of generated embeddings =  (1, 233, 256)
Shape of generated embeddings =  (1, 82, 256)
Shape of generated embeddings =  (1, 71, 256)
Shape of generated embeddings =  (1, 117, 256)
Shape of generated embeddings =  (1, 80, 256)
Shape of generated embeddings =  (1, 190, 256)
Shape of generated embe

Shape of generated embeddings =  (1, 81, 256)
Shape of generated embeddings =  (1, 20, 256)
Shape of generated embeddings =  (1, 48, 256)
Shape of generated embeddings =  (1, 61, 256)
Shape of generated embeddings =  (1, 73, 256)
Shape of generated embeddings =  (1, 44, 256)
Shape of generated embeddings =  (1, 34, 256)
Shape of generated embeddings =  (1, 341, 256)
Shape of generated embeddings =  (1, 29, 256)
Shape of generated embeddings =  (1, 127, 256)
Shape of generated embeddings =  (1, 9, 256)
Shape of generated embeddings =  (1, 272, 256)
Shape of generated embeddings =  (1, 49, 256)
Shape of generated embeddings =  (1, 108, 256)
Shape of generated embeddings =  (1, 123, 256)
Shape of generated embeddings =  (1, 153, 256)
Shape of generated embeddings =  (1, 260, 256)
Shape of generated embeddings =  (1, 57, 256)
Shape of generated embeddings =  (1, 148, 256)
Shape of generated embeddings =  (1, 165, 256)
Shape of generated embeddings =  (1, 30, 256)
Shape of generated embeddi

In [20]:
except_test_lst

[]